# 第 12 章：裝飾器（Decorator）

本章節詳細說明 Python 的裝飾器，這是一種強大的語法糖，用於修改或增強函式和類別的行為。

## 學習目標

- 理解裝飾器的本質與運作原理
- 學會撰寫基本裝飾器和帶參數的裝飾器
- 掌握常用內建裝飾器的使用
- 了解類別裝飾器的應用場景

---

## 12.1 裝飾器基礎

### 什麼是裝飾器？

裝飾器是一個接受函式作為參數並回傳新函式的函式。它允許在不修改原函式程式碼的情況下，增加額外的功能。

In [ ]:
# 裝飾器本質上是一個高階函式
def my_decorator(func):
    def wrapper():
        print("函式執行前")
        func()
        print("函式執行後")
    return wrapper

# 手動應用裝飾器
def say_hello():
    print("Hello!")

say_hello = my_decorator(say_hello)
say_hello()

In [ ]:
# 使用 @ 語法糖（等價於上面的寫法）
@my_decorator
def say_goodbye():
    print("Goodbye!")

say_goodbye()

### JavaScript 對照

```javascript
// JavaScript 沒有內建的裝飾器語法，但可以用高階函式實現類似效果
function myDecorator(func) {
    return function() {
        console.log("函式執行前");
        func();
        console.log("函式執行後");
    };
}

const sayHello = myDecorator(function() {
    console.log("Hello!");
});

sayHello();

// JavaScript 裝飾器提案（Stage 3）
// @myDecorator
// function sayHello() { ... }
```

### 基本裝飾器結構

In [ ]:
def decorator(func):
    """基本裝飾器模板"""
    def wrapper(*args, **kwargs):
        # 前置處理
        result = func(*args, **kwargs)  # 呼叫原函式
        # 後置處理
        return result
    return wrapper

@decorator
def my_function():
    pass

### 處理函式參數

In [ ]:
def log_call(func):
    """記錄函式呼叫"""
    def wrapper(*args, **kwargs):
        print(f"呼叫 {func.__name__}")
        print(f"  參數：args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"  回傳：{result}")
        return result
    return wrapper

@log_call
def add(a, b):
    return a + b

@log_call
def greet(name, greeting="Hello"):
    return f"{greeting}, {name}!"

add(3, 5)
print()
greet("Alice", greeting="Hi")

### 保留原函式資訊（functools.wraps）

裝飾器會改變函式的元資料，使用 `@wraps` 可以保留原函式的資訊。

In [ ]:
from functools import wraps

# 不使用 @wraps 的問題
def bad_decorator(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@bad_decorator
def greet_bad(name):
    """問候函式"""
    return f"Hello, {name}!"

print(f"函式名稱：{greet_bad.__name__}")  # wrapper（錯誤！）
print(f"文件字串：{greet_bad.__doc__}")   # None（錯誤！）

In [ ]:
# 使用 @wraps 保留原函式資訊
def good_decorator(func):
    @wraps(func)  # 保留原函式的 __name__, __doc__ 等
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@good_decorator
def greet_good(name):
    """問候函式"""
    return f"Hello, {name}!"

print(f"函式名稱：{greet_good.__name__}")  # greet_good（正確）
print(f"文件字串：{greet_good.__doc__}")   # 問候函式（正確）

---

## 12.2 實用裝飾器範例

### 計時裝飾器

In [ ]:
import time
from functools import wraps

def timer(func):
    """測量函式執行時間"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"{func.__name__} 執行時間：{end - start:.4f} 秒")
        return result
    return wrapper

@timer
def slow_function():
    time.sleep(0.5)
    return "完成"

result = slow_function()

### 快取裝飾器

In [ ]:
from functools import wraps

def cache(func):
    """簡單的快取裝飾器"""
    cached_results = {}
    
    @wraps(func)
    def wrapper(*args):
        if args in cached_results:
            print(f"從快取取得 {args}")
            return cached_results[args]
        result = func(*args)
        cached_results[args] = result
        return result
    return wrapper

@cache
def fibonacci(n):
    """計算費氏數列"""
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(10))
print(fibonacci(10))  # 從快取取得

In [ ]:
# Python 內建的快取裝飾器（推薦使用）
from functools import lru_cache

@lru_cache(maxsize=128)
def fibonacci_cached(n):
    if n < 2:
        return n
    return fibonacci_cached(n - 1) + fibonacci_cached(n - 2)

print(fibonacci_cached(30))
print(fibonacci_cached.cache_info())

### 重試裝飾器

In [ ]:
import time
import random
from functools import wraps

def retry(max_attempts=3, delay=1):
    """重試裝飾器"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            for attempt in range(1, max_attempts + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    print(f"第 {attempt} 次嘗試失敗：{e}")
                    if attempt < max_attempts:
                        time.sleep(delay)
            raise last_exception
        return wrapper
    return decorator

@retry(max_attempts=3, delay=0.1)
def unstable_api_call():
    if random.random() < 0.7:
        raise ConnectionError("連線失敗")
    return "成功"

# 會自動重試最多 3 次
try:
    result = unstable_api_call()
    print(f"結果：{result}")
except ConnectionError as e:
    print(f"最終失敗：{e}")

### 驗證裝飾器

In [ ]:
from functools import wraps
import inspect

def validate_types(**expected_types):
    """型別驗證裝飾器"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # 取得函式參數名稱
            sig = inspect.signature(func)
            bound = sig.bind(*args, **kwargs)
            bound.apply_defaults()
            
            # 驗證型別
            for param, expected in expected_types.items():
                if param in bound.arguments:
                    value = bound.arguments[param]
                    if not isinstance(value, expected):
                        raise TypeError(
                            f"參數 '{param}' 應為 {expected.__name__}，"
                            f"但收到 {type(value).__name__}"
                        )
            return func(*args, **kwargs)
        return wrapper
    return decorator

@validate_types(name=str, age=int)
def create_user(name, age):
    return {"name": name, "age": age}

print(create_user("Alice", 30))  # 正常

try:
    create_user("Alice", "30")  # 型別錯誤
except TypeError as e:
    print(f"錯誤：{e}")

### 日誌裝飾器

In [ ]:
import logging
from functools import wraps

# 設定日誌
logging.basicConfig(
    level=logging.DEBUG,
    format='%(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def log(level=logging.INFO):
    """日誌裝飾器"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            logger.log(level, f"呼叫 {func.__name__} args={args} kwargs={kwargs}")
            try:
                result = func(*args, **kwargs)
                logger.log(level, f"{func.__name__} 回傳 {result}")
                return result
            except Exception as e:
                logger.exception(f"{func.__name__} 發生例外：{e}")
                raise
        return wrapper
    return decorator

@log(level=logging.DEBUG)
def process_data(data):
    return data.upper()

process_data("hello world")

---

## 12.3 帶參數的裝飾器

### 三層函式結構

帶參數的裝飾器需要三層函式：
1. 外層：接收裝飾器參數
2. 中層：接收被裝飾的函式
3. 內層：實際的 wrapper 函式

In [ ]:
from functools import wraps

def repeat(times):
    """重複執行裝飾器"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            results = []
            for _ in range(times):
                results.append(func(*args, **kwargs))
            return results
        return wrapper
    return decorator

@repeat(times=3)
def say_hello():
    print("Hello!")
    return "done"

results = say_hello()
print(f"結果：{results}")

In [ ]:
# 等價於以下寫法：
# say_hello = repeat(times=3)(say_hello)

def say_hi():
    print("Hi!")
    return "hi"

decorated = repeat(times=2)(say_hi)
decorated()

### 可選參數的裝飾器

讓裝飾器可以同時支援 `@debug` 和 `@debug(prefix="INFO")` 兩種用法。

In [ ]:
from functools import wraps

def debug(func=None, *, prefix="DEBUG"):
    """支援可選參數的裝飾器"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(f"[{prefix}] 呼叫 {func.__name__}")
            return func(*args, **kwargs)
        return wrapper
    
    if func is not None:
        # 不帶參數呼叫：@debug
        return decorator(func)
    # 帶參數呼叫：@debug(prefix="INFO")
    return decorator

# 兩種用法都可以
@debug
def func1():
    return "func1"

@debug(prefix="INFO")
def func2():
    return "func2"

func1()
func2()

### 使用 class 實作裝飾器

In [ ]:
from functools import wraps

class CountCalls:
    """計算函式被呼叫次數的裝飾器（使用類別實作）"""
    
    def __init__(self, func):
        wraps(func)(self)
        self.func = func
        self.count = 0
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"{self.func.__name__} 已被呼叫 {self.count} 次")
        return self.func(*args, **kwargs)

@CountCalls
def say_hello():
    print("Hello!")

say_hello()
say_hello()
say_hello()
print(f"總呼叫次數：{say_hello.count}")

### 帶參數的類別裝飾器

In [ ]:
from functools import wraps
import random

class Retry:
    """重試裝飾器（類別版本）"""
    
    def __init__(self, max_attempts=3, exceptions=(Exception,)):
        self.max_attempts = max_attempts
        self.exceptions = exceptions
    
    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            for attempt in range(1, self.max_attempts + 1):
                try:
                    return func(*args, **kwargs)
                except self.exceptions as e:
                    last_exception = e
                    print(f"嘗試 {attempt}/{self.max_attempts} 失敗：{e}")
            raise last_exception
        return wrapper

@Retry(max_attempts=5, exceptions=(ConnectionError, TimeoutError))
def fetch_data():
    if random.random() < 0.6:
        raise ConnectionError("連線失敗")
    return "資料"

try:
    result = fetch_data()
    print(f"取得：{result}")
except (ConnectionError, TimeoutError) as e:
    print(f"最終失敗：{e}")

---

## 12.4 多重裝飾器

### 堆疊裝飾器

多個裝飾器可以堆疊使用，執行順序是從下到上包裝，從外到內執行。

In [ ]:
from functools import wraps

def bold(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return f"<b>{func(*args, **kwargs)}</b>"
    return wrapper

def italic(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return f"<i>{func(*args, **kwargs)}</i>"
    return wrapper

def underline(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return f"<u>{func(*args, **kwargs)}</u>"
    return wrapper

@bold
@italic
@underline
def greet(name):
    return f"Hello, {name}!"

print(greet("Alice"))
# 等價於：bold(italic(underline(greet)))

### 執行順序示意

In [ ]:
from functools import wraps

def decorator_a(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("A - 前")
        result = func(*args, **kwargs)
        print("A - 後")
        return result
    return wrapper

def decorator_b(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("B - 前")
        result = func(*args, **kwargs)
        print("B - 後")
        return result
    return wrapper

@decorator_a
@decorator_b
def my_function():
    print("函式執行")

my_function()

---

## 12.5 類別裝飾器

裝飾器不僅可以裝飾函式，也可以裝飾類別。

### 單例模式裝飾器

In [ ]:
def singleton(cls):
    """單例模式裝飾器"""
    instances = {}
    
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    
    return get_instance

@singleton
class Database:
    def __init__(self):
        print("建立資料庫連線")
        self.connected = True

db1 = Database()
db2 = Database()  # 不會再次建立
print(f"db1 is db2: {db1 is db2}")

### 為類別添加方法

In [ ]:
def add_repr(cls):
    """自動產生 __repr__ 方法"""
    def __repr__(self):
        attrs = ", ".join(
            f"{k}={v!r}"
            for k, v in self.__dict__.items()
        )
        return f"{cls.__name__}({attrs})"
    
    cls.__repr__ = __repr__
    return cls

@add_repr
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

person = Person("Alice", 30)
print(person)

### 註冊類別（插件系統）

In [ ]:
# 插件註冊系統
registry = {}

def register(cls):
    """註冊類別到 registry"""
    registry[cls.__name__] = cls
    return cls

@register
class PluginA:
    def run(self):
        return "Plugin A 執行中"

@register
class PluginB:
    def run(self):
        return "Plugin B 執行中"

print(f"已註冊的插件：{list(registry.keys())}")

# 動態取得並使用
plugin = registry["PluginA"]()
print(plugin.run())

---

## 12.6 常用內建裝飾器

### @property - 屬性存取器

In [ ]:
import math

class Circle:
    def __init__(self, radius):
        self._radius = radius
    
    @property
    def radius(self):
        """getter"""
        return self._radius
    
    @radius.setter
    def radius(self, value):
        """setter"""
        if value < 0:
            raise ValueError("半徑不能為負數")
        self._radius = value
    
    @property
    def area(self):
        """唯讀屬性"""
        return math.pi * self._radius ** 2

circle = Circle(5)
print(f"半徑：{circle.radius}")
print(f"面積：{circle.area:.2f}")

circle.radius = 10
print(f"新半徑：{circle.radius}")

try:
    circle.area = 100  # 唯讀屬性
except AttributeError as e:
    print(f"錯誤：無法設定唯讀屬性")

### @staticmethod 和 @classmethod

In [ ]:
class MyClass:
    class_var = "類別變數"
    
    def __init__(self, value):
        self.instance_var = value
    
    def instance_method(self):
        """實例方法：可存取實例和類別"""
        return f"instance: {self.instance_var}, class: {self.class_var}"
    
    @classmethod
    def class_method(cls):
        """類別方法：只能存取類別"""
        return f"class: {cls.class_var}"
    
    @classmethod
    def create_default(cls):
        """工廠方法"""
        return cls("default")
    
    @staticmethod
    def static_method(x, y):
        """靜態方法：不能存取實例或類別"""
        return x + y

# 使用
obj = MyClass("實例值")
print(obj.instance_method())
print(MyClass.class_method())
print(MyClass.static_method(1, 2))

default_obj = MyClass.create_default()
print(f"預設實例：{default_obj.instance_var}")

### @functools.lru_cache - LRU 快取

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=128)
def fibonacci(n):
    """帶快取的費氏數列"""
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(f"fibonacci(100) = {fibonacci(100)}")

# 查看快取資訊
print(fibonacci.cache_info())

# 清除快取
fibonacci.cache_clear()
print("快取已清除")
print(fibonacci.cache_info())

### @functools.cache（Python 3.9+）

In [ ]:
from functools import cache

@cache  # 等同於 @lru_cache(maxsize=None)
def factorial(n):
    return n * factorial(n - 1) if n else 1

print(f"10! = {factorial(10)}")
print(f"20! = {factorial(20)}")

### @functools.cached_property（Python 3.8+）

In [ ]:
from functools import cached_property

class DataAnalyzer:
    def __init__(self, data):
        self.data = data
    
    @cached_property
    def statistics(self):
        """只計算一次的屬性"""
        print("計算統計資料...")
        return {
            "mean": sum(self.data) / len(self.data),
            "min": min(self.data),
            "max": max(self.data),
        }

analyzer = DataAnalyzer([1, 2, 3, 4, 5])
print(analyzer.statistics)  # 計算統計資料
print(analyzer.statistics)  # 不再計算，直接回傳

### @functools.total_ordering

In [ ]:
from functools import total_ordering

@total_ordering
class Student:
    """只需定義 __eq__ 和一個比較方法"""
    
    def __init__(self, name, grade):
        self.name = name
        self.grade = grade
    
    def __eq__(self, other):
        return self.grade == other.grade
    
    def __lt__(self, other):
        return self.grade < other.grade
    
    # @total_ordering 會自動產生 __le__, __gt__, __ge__

alice = Student("Alice", 85)
bob = Student("Bob", 90)

print(f"alice < bob: {alice < bob}")
print(f"alice <= bob: {alice <= bob}")
print(f"alice > bob: {alice > bob}")
print(f"alice >= bob: {alice >= bob}")
print(f"alice == bob: {alice == bob}")

### @dataclass（Python 3.7+）

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Point:
    x: float
    y: float

@dataclass
class Person:
    name: str
    age: int
    email: str = ""
    tags: list = field(default_factory=list)

# 自動產生 __init__, __repr__, __eq__
p1 = Point(1.0, 2.0)
p2 = Point(1.0, 2.0)
print(f"p1: {p1}")
print(f"p1 == p2: {p1 == p2}")

person = Person("Alice", 30)
print(f"person: {person}")

In [ ]:
# 更多 dataclass 選項
from dataclasses import dataclass

@dataclass(frozen=True)  # 不可變
class ImmutablePoint:
    x: float
    y: float

point = ImmutablePoint(1.0, 2.0)
try:
    point.x = 3.0  # 會報錯
except Exception as e:
    print(f"錯誤：{type(e).__name__} - 不可變物件")

@dataclass(order=True)  # 可排序
class SortablePerson:
    name: str
    age: int

people = [
    SortablePerson("Bob", 25),
    SortablePerson("Alice", 30),
    SortablePerson("Alice", 20),
]
print(f"排序後：{sorted(people)}")

### @contextlib.contextmanager

In [ ]:
from contextlib import contextmanager
import time

@contextmanager
def timer(name):
    """計時 context manager"""
    start = time.time()
    try:
        yield  # 這裡執行 with 區塊的程式碼
    finally:
        elapsed = time.time() - start
        print(f"{name}: {elapsed:.4f} 秒")

with timer("處理資料"):
    # 模擬耗時操作
    time.sleep(0.2)
    print("處理中...")

In [ ]:
import tempfile
import os

@contextmanager
def temporary_file(content):
    """臨時檔案 context manager"""
    # 建立臨時檔案
    fd, path = tempfile.mkstemp(suffix='.txt')
    try:
        with os.fdopen(fd, 'w') as f:
            f.write(content)
        yield path
    finally:
        # 清理臨時檔案
        if os.path.exists(path):
            os.remove(path)
            print(f"已刪除臨時檔案：{path}")

with temporary_file("Hello, World!") as filepath:
    print(f"臨時檔案路徑：{filepath}")
    with open(filepath, 'r') as f:
        print(f"檔案內容：{f.read()}")

---

## 12.7 裝飾器最佳實踐

### 1. 總是使用 @wraps

In [ ]:
from functools import wraps

def my_decorator(func):
    @wraps(func)  # 保留原函式資訊
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def example():
    """範例函式"""
    pass

print(f"函式名稱：{example.__name__}")
print(f"文件字串：{example.__doc__}")

### 2. 處理所有參數

In [ ]:
def flexible_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):  # 接受任意參數
        print(f"args: {args}, kwargs: {kwargs}")
        return func(*args, **kwargs)
    return wrapper

@flexible_decorator
def flexible_function(a, b, c=3, *args, **kwargs):
    return a + b + c

flexible_function(1, 2, c=10, extra="value")

### 3. 保持裝飾器簡單（單一職責）

In [ ]:
# 好：單一職責，可組合
from functools import wraps
import time

def timer_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        print(f"執行時間：{time.perf_counter() - start:.4f}秒")
        return result
    return wrapper

def log_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"呼叫 {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

# 組合使用
@timer_decorator
@log_decorator
def process():
    time.sleep(0.1)
    return "完成"

process()

### 4. 文件化裝飾器

In [ ]:
from functools import wraps

def retry(max_attempts=3, delay=1.0, exceptions=(Exception,)):
    """
    重試裝飾器。
    
    當函式拋出指定例外時，自動重試。
    
    Args:
        max_attempts: 最大嘗試次數
        delay: 每次重試間隔（秒）
        exceptions: 要捕捉的例外類型
    
    Example:
        @retry(max_attempts=5, delay=0.5)
        def unstable_api():
            ...
    
    Raises:
        最後一次捕捉到的例外
    """
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # 實作省略
            pass
        return wrapper
    return decorator

# 查看文件
print(retry.__doc__)

---

## 練習題

### 練習 1：效能分析裝飾器

寫一個裝飾器，印出函式的執行時間和記憶體使用量。

In [ ]:
# 練習：效能分析裝飾器
import time
import tracemalloc
from functools import wraps

def profile(func):
    """
    效能分析裝飾器
    印出執行時間和記憶體增量
    """
    # 你的程式碼
    pass

@profile
def process_data():
    return [i ** 2 for i in range(100000)]

# 測試
# result = process_data()

In [ ]:
# 解答
import time
import tracemalloc
from functools import wraps

def profile(func):
    """
    效能分析裝飾器
    印出執行時間和記憶體增量
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        # 開始追蹤記憶體
        tracemalloc.start()
        
        # 記錄開始時間
        start_time = time.perf_counter()
        
        # 執行函式
        result = func(*args, **kwargs)
        
        # 計算執行時間
        end_time = time.perf_counter()
        elapsed = end_time - start_time
        
        # 取得記憶體使用量
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        print(f"函式：{func.__name__}")
        print(f"  執行時間：{elapsed:.4f} 秒")
        print(f"  記憶體使用：{current / 1024 / 1024:.2f} MB")
        print(f"  記憶體峰值：{peak / 1024 / 1024:.2f} MB")
        
        return result
    return wrapper

@profile
def process_data():
    return [i ** 2 for i in range(100000)]

result = process_data()

### 練習 2：限流裝飾器

寫一個裝飾器，限制函式的呼叫頻率。

In [ ]:
# 練習：限流裝飾器
import time
from functools import wraps

def rate_limit(calls_per_minute):
    """
    限流裝飾器
    
    @rate_limit(10)  # 每分鐘最多 10 次
    def api_call():
        ...
    """
    # 你的程式碼
    pass

# 測試
# @rate_limit(calls_per_minute=60)  # 每秒 1 次
# def api_call():
#     return "API 回應"

In [ ]:
# 解答
import time
from functools import wraps
from collections import deque

def rate_limit(calls_per_minute):
    """
    限流裝飾器
    
    @rate_limit(10)  # 每分鐘最多 10 次
    def api_call():
        ...
    """
    min_interval = 60.0 / calls_per_minute  # 最小呼叫間隔（秒）
    
    def decorator(func):
        call_times = deque(maxlen=calls_per_minute)
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            now = time.time()
            
            # 移除超過一分鐘的呼叫記錄
            while call_times and now - call_times[0] > 60:
                call_times.popleft()
            
            # 檢查是否超過限制
            if len(call_times) >= calls_per_minute:
                wait_time = 60 - (now - call_times[0])
                raise Exception(f"超過呼叫限制，請等待 {wait_time:.1f} 秒")
            
            # 記錄呼叫時間
            call_times.append(now)
            
            return func(*args, **kwargs)
        return wrapper
    return decorator

@rate_limit(calls_per_minute=3)  # 每分鐘最多 3 次（測試用）
def api_call():
    return "API 回應"

# 測試
for i in range(5):
    try:
        result = api_call()
        print(f"第 {i+1} 次呼叫：{result}")
    except Exception as e:
        print(f"第 {i+1} 次呼叫失敗：{e}")

### 練習 3：JSON 序列化裝飾器

寫一個類別裝飾器，自動為類別添加 JSON 序列化功能。

In [ ]:
# 練習：JSON 序列化裝飾器
import json

def jsonable(cls):
    """
    讓類別可以轉換為 JSON
    
    @jsonable
    class Person:
        def __init__(self, name, age):
            self.name = name
            self.age = age
    
    p = Person("Alice", 30)
    print(p.to_json())  # {"name": "Alice", "age": 30}
    """
    # 你的程式碼
    pass

# 測試
# @jsonable
# class Person:
#     def __init__(self, name, age):
#         self.name = name
#         self.age = age

In [ ]:
# 解答
import json

def jsonable(cls):
    """
    讓類別可以轉換為 JSON
    """
    def to_json(self, indent=None):
        """將物件轉換為 JSON 字串"""
        return json.dumps(self.__dict__, indent=indent, ensure_ascii=False)
    
    def to_dict(self):
        """將物件轉換為字典"""
        return self.__dict__.copy()
    
    @classmethod
    def from_json(cls_method, json_str):
        """從 JSON 字串建立物件"""
        data = json.loads(json_str)
        return cls_method(**data)
    
    @classmethod
    def from_dict(cls_method, data):
        """從字典建立物件"""
        return cls_method(**data)
    
    # 添加方法到類別
    cls.to_json = to_json
    cls.to_dict = to_dict
    cls.from_json = from_json
    cls.from_dict = from_dict
    
    return cls

@jsonable
class Person:
    def __init__(self, name, age, email=""):
        self.name = name
        self.age = age
        self.email = email

# 測試
p1 = Person("Alice", 30, "alice@example.com")
print(f"to_json(): {p1.to_json()}")
print(f"to_dict(): {p1.to_dict()}")

# 從 JSON 建立
json_str = '{"name": "Bob", "age": 25, "email": "bob@example.com"}'
p2 = Person.from_json(json_str)
print(f"from_json(): {p2.name}, {p2.age}")

---

## 小結

### 裝飾器結構

```python
# 無參數裝飾器
def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

# 帶參數裝飾器
def decorator_with_args(arg1, arg2):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        return wrapper
    return decorator

# 類別裝飾器
class Decorator:
    def __init__(self, func):
        wraps(func)(self)
        self.func = func
    
    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)
```

### 常用內建裝飾器

| 裝飾器 | 模組 | 用途 |
|--------|------|------|
| `@property` | 內建 | 將方法變成屬性 |
| `@staticmethod` | 內建 | 靜態方法 |
| `@classmethod` | 內建 | 類別方法 |
| `@lru_cache` | functools | 快取結果 |
| `@cache` | functools | 無限快取 |
| `@cached_property` | functools | 快取屬性 |
| `@total_ordering` | functools | 自動產生比較方法 |
| `@wraps` | functools | 保留函式資訊 |
| `@dataclass` | dataclasses | 自動產生類別方法 |
| `@contextmanager` | contextlib | 建立 context manager |
| `@abstractmethod` | abc | 抽象方法 |

### 裝飾器 vs JavaScript

| Python | JavaScript | 說明 |
|--------|------------|------|
| `@decorator` | 高階函式 | Python 有語法糖 |
| `@decorator(args)` | 高階函式 | 帶參數 |
| 類別裝飾器 | 類別裝飾器提案 | ES 提案中 |
| `@property` | `get/set` | 屬性存取器 |
| `@staticmethod` | `static` | 靜態方法 |